## NOTICIAS VERDADERAS

In [1]:
!pip install selenium
!apt-get update -y
!apt-get install -y chromium-chromedriver



"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin

# Chrome headless mínimo
opt = Options()
opt.add_argument("--headless=new"); opt.add_argument("--no-sandbox"); opt.add_argument("--disable-dev-shm-usage")
d = webdriver.Chrome(options=opt)
w = WebDriverWait(d, 15)




In [3]:
# --- 1) Constantes de origen (dominio base) ---
BBC_BASE = "https://www.bbc.com"

# --- 2) Función para ir siempre a la página base a partir de la <a> dada ---
def go_to_base_page():
    """
    Busca la <a> de 'América Latina', toma su href relativo,
    construye la URL absoluta y navega allí.
    """
    # Primero carga la home de BBC Mundo
    d.get(urljoin(BBC_BASE, "/mundo"))

    # Espera a que exista el enlace de la sección (por su aria-labelledby)
    try:
        anchor = w.until(
            EC.presence_of_element_located((
                By.CSS_SELECTOR,
                "a[aria-labelledby='NavigationLinks-América Latina']"
            ))
        )
    except Exception:
        # Fallback: por texto visible (por si cambia el aria-labelledby)
        anchor = w.until(
            EC.presence_of_element_located((
                By.XPATH,
                "//a[.//span[contains(normalize-space(),'América Latina')]]"
            ))
        )

    # Toma href relativo y construye absoluto
    href_rel = anchor.get_attribute("href")
    # Algunas plantillas devuelven relativo; si fuera completo, urljoin igual lo respeta
    abs_url = urljoin(BBC_BASE, href_rel)

    # Navega a la página base
    d.get(abs_url)

    # Pequeña verificación
    w.until(lambda drv: drv.current_url.startswith(abs_url))
    print(f"[OK] Base cargada: {d.current_url}")

# --- 3) Llamada inicial
go_to_base_page()


[OK] Base cargada: https://www.bbc.com/mundo/topics/c7zp57yyz25t


In [4]:
# --- 6) Paginar y extraer hasta reunir 75 noticias ---
from urllib.parse import urljoin

base_url = d.current_url.rstrip("/")

all_results, seen = [], set()
page = 1
while len(all_results) < 75:
    page_url = base_url if page == 1 else f"{base_url}?page={page}"
    d.get(page_url)
    try:
        w.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "ul[data-testid='topic-promos'] li h2 a")
        ))
    except Exception:
        break  # si no carga la lista, detenemos

    anchors = d.find_elements(By.CSS_SELECTOR, "ul[data-testid='topic-promos'] li h2 a")
    if not anchors:
        break

    for a in anchors:
        title = a.text.strip()
        href = urljoin(BBC_BASE, a.get_attribute("href"))
        if title and href and href not in seen:
            all_results.append((title, href))
            seen.add(href)
            if len(all_results) >= 75:
                break
    page += 1

# Visitar cada noticia (máximo 75) y extraer cuerpo en párrafos
article_texts = []
for title, href in all_results[:75]:
    d.get(href)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main[role='main']")))
    ps = d.find_elements(
        By.XPATH,
        "//main[@role='main']//div[@dir='ltr']/p"
        "[not(ancestor::figure)]"
        "[not(ancestor::section[@data-e2e='recommendations-heading'])]"
        "[not(ancestor::section[@aria-label='Publicidad 2'])]"
    )
    body = "\n\n".join(p.text.strip() for p in ps if p.text and p.text.strip())
    print("="*80)
    print(f"Título: {title}")
    print(f"URL: {href}\n")
    print(body[:100])
    print("\n")
    article_texts.append((title, href, body))


Título: Encuentran muertos en México a los músicos colombianos B-King y DJ Regio Clown tras varios días desaparecidos
URL: https://www.bbc.com/mundo/articles/ckgzkjzxnkwo

Las autoridades confirmaron que los músicos colombianos Bayron Sánchez Salazar y Jorge Luis Herrera 


Título: El inédito respaldo que el gobierno de Trump le ofrece a Milei en medio del deterioro de la economía argentina
URL: https://www.bbc.com/mundo/articles/cp8wy58n480o

En un sorpresivo anuncio, el secretario del Tesoro de Estados Unidos, Scott Bessent, se comprometió 


Título: "Si tengo que dejar mi vida luchando, la dejo": el entrenamiento a civiles que hace el gobierno de Venezuela ante la "guerra no declarada" de EE.UU.
URL: https://www.bbc.com/mundo/articles/c98enp0n8yro

"Estoy presto para presentarme cuando sea el llamado. Tenemos que salir a defender la patria".

Edit


Título: Las vecinas que consiguieron que su pequeño pueblo al norte de Inglaterra se hermanara con Machu Picchu
URL: https://www.bbc.co

In [5]:
len(article_texts)

75

In [6]:
import os
import re

# --- 7) Guardar cada noticia como archivo .txt ---
output_dir = "/content/noticias_bbc"
os.makedirs(output_dir, exist_ok=True)

for idx, (title, href, body) in enumerate(article_texts, start=1):
    # Limpiar el título para usarlo como nombre de archivo
    safe_title = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ ]', '', title)  # quitar símbolos raros
    safe_title = "_".join(safe_title.split())                     # espacios -> _
    filename = f"{idx:02d}_{safe_title[:50]}.txt"                 # limitar longitud

    filepath = os.path.join(output_dir, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(title + "\n\n")
        f.write(body)

print(f"[OK] Archivos creados en: {output_dir}")


[OK] Archivos creados en: /content/noticias_bbc


In [7]:
!zip -r noticias_bbc.zip /content/noticias_bbc


  adding: content/noticias_bbc/ (192 bytes security) (stored 0%)
  adding: content/noticias_bbc/01_Encuentran_muertos_en_M�xico_a_los_m�sicos_colombi.txt (172 bytes security) (deflated 56%)
  adding: content/noticias_bbc/02_El_in�dito_respaldo_que_el_gobierno_de_Trump_le_of.txt (172 bytes security) (deflated 55%)
  adding: content/noticias_bbc/03_Si_tengo_que_dejar_mi_vida_luchando_la_dejo_el_ent.txt (172 bytes security) (deflated 55%)
  adding: content/noticias_bbc/04_Las_vecinas_que_consiguieron_que_su_peque�o_pueblo.txt (172 bytes security) (deflated 52%)
  adding: content/noticias_bbc/05_Del_zapato_de_Kruschev_al_huele_a_azufre_de_Ch�vez.txt (172 bytes security) (deflated 55%)
  adding: content/noticias_bbc/06_Por_qu�_los_ataques_militares_de_EEUU_a_barcos_de_.txt (172 bytes security) (deflated 56%)
  adding: content/noticias_bbc/07_No_corro_no_grito_no_empujo_qu�_cambi�_en_Ciudad_d.txt (172 bytes security) (deflated 59%)
  adding: content/noticias_bbc/08_La_presencia_militar_de_EE

In [9]:
# from google.colab import files
# files.download("noticias_bbc.zip")
